In [1]:
import pandas as pd
TrainingLabels = pd.read_csv('Training labels.csv', index_col='id')
TrainingFeatures = pd.read_csv('Training features.csv', index_col='id')
TestFeatures = pd.read_csv('Test features.csv', index_col='id')



In [2]:
preprocess = pd.concat([TrainingFeatures,TestFeatures])
#preprocess

In [3]:
preprocess.drop(['wpt_name','num_private','subvillage','region','lga','ward','recorded_by','scheme_name',
'extraction_type_class','extraction_type','payment_type','water_quality','quantity_group','source_type',
'source','waterpoint_type','date_recorded', 'longitude','latitude','management_group'],axis=1,inplace=True)
preprocess.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin',
       'region_code', 'district_code', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year',
       'extraction_type_group', 'management', 'payment', 'quality_group',
       'quantity', 'source_class', 'waterpoint_type_group'],
      dtype='object')

In [4]:
import numpy as nan
preprocess['funder'] = preprocess['funder'].str.lower()
preprocess['funder'].fillna(preprocess.installer, inplace=True)
preprocess['funder'].fillna('other', inplace=True)
funder_reduction = ['kkkt','vision','foundat','tlc','wsd','norad','sema','wfp','ded','unicef','danida','hes','rws','dhv','isf']

for category in funder_reduction:
    subCategory = preprocess[preprocess['funder'].str.contains(category)]
    unique = subCategory['funder'].unique()
    preprocess['funder'].replace(to_replace=unique,value=category, inplace=True)

multi = [
[['hifab','amref','oxfa','wateraid','Wate Aid','Water Aid','adra','plan','Ces(gmbh)','Ces (gmbh)','shipo','dmdd','oikos'],'project'],
[['mzee','priva','mwita','ltd','bank','kanis'],'private'],
[['municipal council', 'distric','govern','ministry','supply','adb','rudep'],'govern'],
[['rc','roman',' mission','cathoric','st ','churc','anglican','tcrs'],'church'],
[['total land care','total landcare','totaland care','totoland','totoland care'],'totallandcare'],
[['tassaf','tasaf'],'tasaf'],
[['ngo ',' ngo'],'ngo'],
[['rotary','club'],'club'],
[['islam','muslim'],'muslim'],
[['said ',' said'],'said'],
[['british','japan','egypt','belgian','italy','iran','china','finland','irish','swedish','german','netherlan','european','kuwait','african','jaica','jica'],'foreign'],
[['village','community','rural','halmashauri','lga'],'village'],
[['lga','adb'],'lga & adb'],
[['Fin Water','Fini Water','Finn Water','Finw','Finwater','Fiwater'],'fini']
]
for category in multi:
    for name in category[0]:
        subCategory = preprocess.loc[preprocess['funder'].str.contains(name)]
        unique = subCategory['funder'].unique()
        preprocess['funder'].replace(to_replace=unique,value=category[1], inplace=True)

preprocess['funder'].replace(to_replace=('mission','missio','missi'),value='church', inplace=True)

preprocess['funder'].replace(to_replace=('0', 'nan', '-', ''), value='other', inplace=True)
count = preprocess.groupby('funder')['funder'].count()
other_list = count[count<500].index.tolist()
preprocess['funder'].replace(to_replace=other_list, value='other', inplace=True)  

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(preprocess.groupby('funder')['funder'].count().sort_values(ascending=False))

C:\Users\Yasith\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


funder
other      24819
govern     15431
danida      3921
project     3661
private     3402
foreign     3163
church      3077
hes         2802
kkkt        1953
rws         1742
vision      1603
village     1430
unicef      1350
tasaf       1316
dhv         1146
norad       1078
dwsp        1012
water        739
dwe          605
Name: funder, dtype: int64


In [5]:
#pd.crosstab(index=preprocess.scheme_management,columns=preprocess.management)
preprocess['management_cross'] = preprocess['scheme_management'] + preprocess['management']
count = preprocess.groupby('management_cross')['management_cross'].count()
other_list = count[count<500].index.tolist()
preprocess['management_cross'].replace(to_replace=other_list, value='other', inplace=True)  
preprocess.drop(['scheme_management','management'],axis=1,inplace=True)
#preprocess.groupby('management_cross')['management_cross'].count()


In [6]:
import numpy as np

preprocess['gps_height'] = preprocess['gps_height'].abs()

preprocess['construction_year'].replace(to_replace=0, value=np.nan, inplace=True)
preprocess['age'] = 2013 - preprocess.construction_year
preprocess.drop('construction_year', axis = 1, inplace = True)

pop_bins = [-1,0,50, 200, 31000]
pop_labels = [1,2,3,4]
TrainingFeatures['pop_binned'] = pd.cut(TrainingFeatures['population'], bins=pop_bins, labels=pop_labels)
preprocess.drop('population', axis = 1, inplace = True)

preprocess.drop('installer', axis = 1, inplace = True)

bool_mapper = {"False":0, "True":1}
preprocess["permit"] = preprocess["permit"].replace(bool_mapper)
preprocess["public_meeting"] = preprocess["public_meeting"].replace(bool_mapper)


In [7]:
preprocess.isna().sum()

amount_tsh                   0
funder                       0
gps_height                   0
basin                        0
region_code                  0
district_code                0
public_meeting            4155
permit                    3793
extraction_type_group        0
payment                      0
quality_group                0
quantity                     0
source_class                 0
waterpoint_type_group        0
management_cross          4846
age                      25969
dtype: int64

In [8]:
preprocess["permit"] = preprocess["permit"].astype('float')
preprocess["public_meeting"] = preprocess["public_meeting"].astype('float')

In [9]:
object_columns = preprocess.select_dtypes(include=['object']).columns
for object_col in object_columns:
    preprocess[object_col] = preprocess[object_col].astype('category')

In [10]:
preprocess.dtypes

amount_tsh                float64
funder                   category
gps_height                  int64
basin                    category
region_code                 int64
district_code               int64
public_meeting            float64
permit                    float64
extraction_type_group    category
payment                  category
quality_group            category
quantity                 category
source_class             category
waterpoint_type_group    category
management_cross         category
age                       float64
dtype: object

In [11]:
from category_encoders.wrapper import PolynomialWrapper
from category_encoders.cat_boost import CatBoostEncoder

cbe = CatBoostEncoder()
wrap = PolynomialWrapper(cbe)
wrap.fit(preprocess.iloc[:59400,:],TrainingLabels)
preprocess = wrap.transform(preprocess)


C:\Users\Yasith\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [12]:
from sklearn.impute import KNNImputer

imputer = KNNImputer()
preprocess_array = imputer.fit_transform(preprocess)


In [13]:
preprocess = pd.DataFrame(preprocess_array, index=preprocess.index, columns=preprocess.columns)
preprocess.isna().sum()

amount_tsh                                       0
gps_height                                       0
region_code                                      0
district_code                                    0
public_meeting                                   0
permit                                           0
age                                              0
funder_non functional                            0
basin_non functional                             0
extraction_type_group_non functional             0
payment_non functional                           0
quality_group_non functional                     0
quantity_non functional                          0
source_class_non functional                      0
waterpoint_type_group_non functional             0
management_cross_non functional                  0
funder_functional needs repair                   0
basin_functional needs repair                    0
extraction_type_group_functional needs repair    0
payment_functional needs repair

In [14]:
preprocess["permit"] = preprocess["permit"].astype('boolean')
preprocess["public_meeting"] = preprocess["public_meeting"].astype('boolean')

TypeError: Need to pass bool-like values

In [ ]:
# import matplotlib.pyplot as plt

# import seaborn as sns
# from sklearn.feature_selection import mutual_info_regression


# # Set Matplotlib defaults
# plt.style.use("seaborn-whitegrid")
# plt.rc("figure", autolayout=True)
# plt.rc(
#     "axes",
#     labelweight="bold",
#     labelsize="large",
#     titleweight="bold",
#     titlesize=14,
#     titlepad=10,
# )

# def make_mi_scores(X, y):
#     #X = X.copy()
#     #for colname in X.select_dtypes(["object", "category"]):
#     #    X[colname], _ = X[colname].factorize()
#     # All discrete features should now have integer dtypes
#     discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
#     mi_scores = mutual_info_regression(X, y)
#     mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
#     return mi_scores


# def plot_mi_scores(scores):
#     scores = scores.sort_values(ascending=True)
#     width = np.arange(len(scores))
#     ticks = list(scores.index)
#     plt.barh(width, scores)
#     plt.yticks(width, ticks)
#     plt.title("Mutual Information Scores")

# mi_scores = make_mi_scores(preprocess.iloc[:59400,:],TrainingLabels)

# print(mi_scores)

# plt.figure(dpi=100, figsize=(8, 5))
# plot_mi_scores(mi_scores)

C:\Users\Yasith\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: could not convert string to float: 'functional'

In [15]:
TrainingFeatures = preprocess.iloc[:59400,:]
TestFeatures = preprocess.iloc[59400:,:]

TrainingFeatures.to_csv('training_cleaned_knn.csv', index=True)
TestFeatures.to_csv('test_cleaned_knn.csv', index=True)

In [17]:
preprocess.dtypes

amount_tsh                                       float64
gps_height                                       float64
region_code                                      float64
district_code                                    float64
public_meeting                                   float64
permit                                           float64
age                                              float64
funder_non functional                            float64
basin_non functional                             float64
extraction_type_group_non functional             float64
payment_non functional                           float64
quality_group_non functional                     float64
quantity_non functional                          float64
source_class_non functional                      float64
waterpoint_type_group_non functional             float64
management_cross_non functional                  float64
funder_functional needs repair                   float64
basin_functional needs repair  

In [18]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

cat = CatBoostClassifier()
#print (cross_val_score(cat, TrainingFeatures, TrainingLabels))
train_X, val_X, train_y, val_y = train_test_split(TrainingFeatures, TrainingLabels, random_state=1)
cat.fit(TrainingFeatures, TrainingLabels)


Learning rate set to 0.097451
0:	learn: 1.0289119	total: 266ms	remaining: 4m 25s
1:	learn: 0.9731240	total: 349ms	remaining: 2m 54s
2:	learn: 0.9288385	total: 378ms	remaining: 2m 5s
3:	learn: 0.8929511	total: 401ms	remaining: 1m 39s
4:	learn: 0.8624260	total: 429ms	remaining: 1m 25s
5:	learn: 0.8380173	total: 444ms	remaining: 1m 13s
6:	learn: 0.8155943	total: 463ms	remaining: 1m 5s
7:	learn: 0.7966283	total: 481ms	remaining: 59.6s
8:	learn: 0.7809056	total: 503ms	remaining: 55.4s
9:	learn: 0.7667115	total: 528ms	remaining: 52.3s
10:	learn: 0.7545710	total: 548ms	remaining: 49.2s
11:	learn: 0.7434927	total: 572ms	remaining: 47.1s
12:	learn: 0.7328881	total: 594ms	remaining: 45.1s
13:	learn: 0.7243981	total: 613ms	remaining: 43.2s
14:	learn: 0.7162325	total: 637ms	remaining: 41.8s
15:	learn: 0.7095251	total: 657ms	remaining: 40.4s
16:	learn: 0.7026687	total: 673ms	remaining: 38.9s
17:	learn: 0.6967629	total: 694ms	remaining: 37.9s
18:	learn: 0.6923492	total: 714ms	remaining: 36.9s
19:	le

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(cat, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

Weight,Feature
0.0599 ± 0.0064,quantity_non functional
0.0445 ± 0.0057,installer_non functional
0.0216 ± 0.0026,quantity_functional needs repair
0.0209 ± 0.0025,age
0.0196 ± 0.0040,latitude
0.0183 ± 0.0030,waterpoint_type_non functional
0.0178 ± 0.0026,longitude
0.0173 ± 0.0035,extraction_type_non functional
0.0121 ± 0.0008,installer_functional needs repair
0.0111 ± 0.0025,gps_height


In [20]:
cat.get_feature_importance()

array([2.47182873, 8.6984214 , 6.30325604, 7.65875622, 1.11141616,
       2.72354922, 8.34672145, 3.65347061, 4.73037959, 5.30525877,
       3.85523788, 0.92586218, 8.82392305, 1.31828763, 3.78836109,
       3.03459321, 3.97661061, 3.70836093, 4.3285781 , 2.75161164,
       0.57575097, 4.93981817, 1.21997006, 2.30946137, 3.44051493])

In [ ]:
s = cat.predict(TestFeatures)
#s = le.inverse_transform(s)
seas_df = pd.DataFrame(s, columns = ['status_group'], index = TestFeatures.index)

seas_df.to_csv('output_yasith_1.csv', index=True)